In [2]:
import tensorflow as tf
import numpy as np
import glob as gl
import pickle
from music21 import note, converter, instrument, chord
from tqdm import tqdm

from keras.utils import np_utils 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint

In [3]:
def noteExtracter():
    note_arr = []

    for file in gl.glob("raw_data/*.mid"):

        midi_file = converter.parse(file)
        print(f"Parsing {file}")

        notes = None
        try:
            var = instrument.partitionByInstrument(midi_file)
            notes = var.parts[0].recurse()
        except:
            notes = midi_file.flat.notes 

        for nt in notes:
            if isinstance(nt, note.Note):
                note_arr.append(str(nt.pitch))
            elif isinstance(nt, chord.Chord):
                note_arr.append('.'.join(str(n) for n in nt.normalOrder))

        with open('data/notes', 'wb') as path:
            pickle.dump(note_arr, path)

    return note_arr

In [4]:
def seqPreparetion(notes, vocab):
    seqLength = 100
    pitches = sorted(set(it for it in notes))
    noteEnum = dict((note, num) for num, note in enumerate(pitches))
    
    networkInput = []
    networkOutput = []
    
    for i in range(0, len(notes) - seqLength, 1):
        seqIN = notes[i : i+seqLength]  
        seqOut = notes[i + seqLength]
        networkInput.append([noteEnum[ch] for ch in seqIN])
        networkOutput.append(noteEnum[seqOut])
    
    patterns = len(networkInput)
    
    networkInput = np.reshape(networkInput, (patterns, seqLength, 1))
    networkOutput = np_utils.to_categorical(networkOutput)
    return(networkInput, networkOutput)

In [5]:
def neuralNetwork(networkIN, vocab):
    model = Sequential()
    model.add(LSTM(512, input_shape=(networkIN.shape[1], networkIN.shape[2]), recurrent_dropout=0.2, return_sequences=True))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    return model

In [6]:
def train(model, netIn, netOut):
    fileName = "weights.hdf5"
    checkpoint = ModelCheckpoint(fileName, monitor='loss', verbose=0, save_best_only=True, mode='min')
    callback = [checkpoint]
    
    model.fit(netIn, netOut, epochs=1, batch_size=128, callbacks=callback)

In [7]:
tf.debugging.set_log_device_placement(True)
notes = noteExtracter()
vocab = len(set(notes))
networkIN, networkOUT = seqPreparetion(notes, len(set(notes)))
model = neuralNetwork(networkIN, vocab)
train(model, networkIN, networkOUT)


Parsing raw_data\album_72_1_(c)yogore.mid
Parsing raw_data\album_72_2_(c)yogore.mid
Parsing raw_data\album_72_3_(c)yogore.mid
Parsing raw_data\album_72_4_(c)yogore.mid
Parsing raw_data\album_72_5_(c)yogore.mid
Parsing raw_data\album_72_6_(c)yogore.mid
Parsing raw_data\allegro_appassionato_70_(c)yogore.mid
Parsing raw_data\danse_macabre_40_(c)yogore.mid
Parsing raw_data\etudes_135-01_(c)yogore.mid
Parsing raw_data\etudes_135-02_(c)yogore.mid
Parsing raw_data\etudes_135-03_(c)yogore.mid
Parsing raw_data\etudes_135-04_(c)yogore.mid
Parsing raw_data\etudes_135-05_(c)yogore.mid
Parsing raw_data\etudes_135-06_(c)yogore.mid
Parsing raw_data\etudes_52-3_(c)yogore.mid
Parsing raw_data\etudes_52-4_(c)yogore.mid
Parsing raw_data\etudes_52-5_(c)yogore.mid
Parsing raw_data\etudes_52-6_(c)yogore.mid
Parsing raw_data\saint-saens_60813a_etude_in_waltz_form_(nc)smythe.mid
Parsing raw_data\saint_saens_canariote_88_(c)yogore.mid
Parsing raw_data\saint_saens_etudes_111_1_(c)yogore.mid
Parsing raw_data\sai

Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ConcatV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0

Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in 

Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:loc

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:loc

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17188057002360935308
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3553368152509528782
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3134364057
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1293756640016128404
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5323697650904735572
physical_device_desc: "device: XLA_GPU device"
]


In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = ""